In [2]:
!pip install torch

  Using cached sympy-1.14.0-py3-none-any.whl.metadata (12 kB)
  Using cached mpmath-1.3.0-py3-none-any.whl.metadata (8.6 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.5/74.5 MB 10.7 MB/s  0:00:06 eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 12.1 MB/s  0:00:00
Using cached sympy-1.14.0-py3-none-any.whl (6.3 MB)
Using cached mpmath-1.3.0-py3-none-any.whl (536 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6/6 [torch]32m5/6 [torch]kx]


In [5]:
!pip install transformers

  Using cached tokenizers-0.22.1-cp39-abi3-macosx_11_0_arm64.whl.metadata (6.8 kB)
  Using cached tqdm-4.67.1-py3-none-any.whl.metadata (57 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 10.1 MB/s  0:00:01ta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 566.1/566.1 kB 11.9 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 12.5 MB/s  0:00:00 eta 0:00:01
Using cached tokenizers-0.22.1-cp39-abi3-macosx_11_0_arm64.whl (2.9 MB)
Using cached tqdm-4.67.1-py3-none-any.whl (78 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7/7 [transformers] [transformers]ub]


In [8]:
! pip install gensim

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.5/24.5 MB 10.6 MB/s  0:00:02 eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.9/20.9 MB 11.3 MB/s  0:00:01 eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4/4 [gensim]2m3/4 [gensim]


In [9]:
! pip install contractions

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4/4 [contractions]


In [7]:
import pandas as pd
import numpy as np
import torch
from transformers import BertTokenizer, BertModel

import contractions
# For progress bars
# from tqdm import tqdm

In [16]:
review_path = "data/raw_reviews.csv"

review_df = pd.read_csv(review_path)[["recommendationid", "Appname", "review_text"]]
review_df.head()

,recommendationid,Appname,review_text
0,212664845,ARC Raiders,Addictive. Stressful. Time waster.
1,212664820,ARC Raiders,fuak arc\r\n
2,212664759,ARC Raiders,W
3,212664729,ARC Raiders,awesome game!
4,212664705,ARC Raiders,If the Steam comments section is like every ot...


In [17]:
review_df.shape

(69261, 3)

In [18]:
review_df.groupby("Appname").count()

,recommendationid,review_text
Appname,,
7 Days to Die,300,299
A Total War Saga: TROY,300,300
ARC Raiders,300,299
ARK: Survival Ascended,300,299
Abiotic Factor,300,300
...,...,...
World of Warships,100,99
Yakuza 0,300,298
iRacing,300,297


# Part 1: Preprocessing

## Contraction Expansion

In [19]:
!pip install contractions
import contractions

In [28]:
# review_df['review_text']
review_df['review_text'] = review_df['review_text'].astype('str')

In [36]:
# review_df.loc((review_df['review_text'].str.len() > 0)).shape
review_df = review_df[
    review_df["review_text"].apply(lambda x: len(x) > 20)
    ]
review_df.shape

(48713, 3)

In [37]:
review_df['review_expanded'] = review_df['review_text'].apply(lambda x:contractions.fix(x).lower())
# review_df.iloc[0,2]

## contraction expansion doesn't work because the apostrophe's are being escaped with \'

IndexError: string index out of range

## Tokenization

In [39]:
!pip install nltk

  Using cached joblib-1.5.2-py3-none-any.whl.metadata (5.6 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 5.9 MB/s  0:00:00 eta 0:00:01
Using cached joblib-1.5.2-py3-none-any.whl (308 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3/3 [nltk]2/3 [nltk]


In [40]:
from nltk.tokenize import wordpunct_tokenize
from nltk.tokenize import word_tokenize

In [41]:
# tokenize each review
review_df['review_tokens'] = review_df['review_text'].apply(wordpunct_tokenize)
review_df['review_tokens']
# review_df.iloc[0,3]

## wordpunct tokenizer is handling the contractions and the tab characters well

0            [Addictive, ., Stressful, ., Time, waster, .]
4        [If, the, Steam, comments, section, is, like, ...
5        [I, like, the, gathering, and, sneaking, aroun...
6        [Very, well, made, game, ,, every, time, I, ho...
7        [I, thought, this, would, be, too, sweaty, for...
                               ...                        
69256                       [We, have, Hades, at, home, :]
69257    [Unfortunately, can, not, recommend, the, game...
69258    [if, you, enjoy, soulstone, survivors, or, had...
69259    [Great, game, overall, !, I, really, enjoy, th...
69260    [The, game, is, pretty, good, for, some, of, t...
Name: review_tokens, Length: 48713, dtype: object

## POS Tagging

In [42]:
import nltk
nltk.download('averaged_perceptron_tagger_eng')

[nltk_data] Downloading package averaged_perceptron_tagger_eng to
[nltk_data]     /Users/joh11678/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger_eng.zip.


True

In [43]:
# Directly apply nltk.postag

%time review_df['review_tagged_NLTK'] = review_df['review_tokens'].apply(nltk.pos_tag)
review_df['review_tagged_NLTK']

CPU times: user 30.1 s, sys: 176 ms, total: 30.2 s
Wall time: 30.3 s


0        [(Addictive, NNP), (., .), (Stressful, NNP), (...
4        [(If, IN), (the, DT), (Steam, NNP), (comments,...
5        [(I, PRP), (like, VBP), (the, DT), (gathering,...
6        [(Very, RB), (well, RB), (made, VBN), (game, N...
7        [(I, PRP), (thought, VBD), (this, DT), (would,...
                               ...                        
69256    [(We, PRP), (have, VBP), (Hades, VBN), (at, IN...
69257    [(Unfortunately, RB), (can, MD), (not, RB), (r...
69258    [(if, IN), (you, PRP), (enjoy, VBP), (soulston...
69259    [(Great, JJ), (game, NN), (overall, JJ), (!, ....
69260    [(The, DT), (game, NN), (is, VBZ), (pretty, RB...
Name: review_tagged_NLTK, Length: 48713, dtype: object

## Lemmatization

In [44]:
## download lemmatizer
from nltk.stem import WordNetLemmatizer
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/joh11678/nltk_data...


True

In [45]:
# function to lemmatize words based on different POS tag types;
def lemmatize_POS (tagged_columns):
  wnl = WordNetLemmatizer()
  text=[]
  for word, tag in tagged_columns:
    if tag.startswith('NN'):
      token = wnl.lemmatize(word, pos='n') # n is NOUN
    elif tag.startswith('VB'):
      token = wnl.lemmatize(word, pos='v') # v is VERB
    elif tag.startswith('JJ'):
      token = wnl.lemmatize(word, pos='a') # a is ADJ
    elif tag.startswith('RB'):
      token = wnl.lemmatize(word, pos='r') # r is ADV
    else:
      token = word
    text.append(token)
  #cleaned_corpus.append(text)
  return text

In [46]:
review_df['review_lemma_NLTK'] = review_df['review_tagged_NLTK'].apply(lemmatize_POS) ## apply lemmatization based on POS tag
review_df['review_lemma_NLTK']

0            [Addictive, ., Stressful, ., Time, waster, .]
4        [If, the, Steam, comment, section, be, like, e...
5        [I, like, the, gathering, and, sneak, around, ...
6        [Very, well, make, game, ,, every, time, I, ho...
7        [I, think, this, would, be, too, sweaty, for, ...
                               ...                        
69256                       [We, have, Hades, at, home, :]
69257    [Unfortunately, can, not, recommend, the, game...
69258    [if, you, enjoy, soulstone, survivor, or, hade...
69259    [Great, game, overall, !, I, really, enjoy, th...
69260    [The, game, be, pretty, good, for, some, of, t...
Name: review_lemma_NLTK, Length: 48713, dtype: object

## Text Normalization

In [47]:
from string import punctuation
nltk.download('stopwords')
nltk.download('punkt')

from nltk.corpus import stopwords
import re
def preprocessing(tokens):
  tokens = [token.lower() for token in tokens] # lowercasing

  tokens = [token for token in tokens if not token.isdigit()] # remove digit - isdigit is a python buit-in method.

  tokens = [token for token in tokens if token not in punctuation] # remove punctuations

  # Remove tokens that are entirely punctuation (single or multiple)
  tokens = [token for token in tokens if not re.fullmatch(r"[^\w]+", token)]

  mystopwords = set(stopwords.words("english")) # use english stopwords list.
  tokens = [token for token in tokens if token not in mystopwords] # remove stopwords

  tokens = [token for token in tokens if len(token)>=3] # remove tokens with one or two characters

  return tokens

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/joh11678/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /Users/joh11678/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [48]:
%time review_df['review_lemma_NLTK_final'] = review_df['review_lemma_NLTK'].apply(preprocessing) ## apply preprocessing to text
review_df['review_lemma_NLTK_final']

CPU times: user 1.86 s, sys: 611 ms, total: 2.47 s
Wall time: 2.47 s


0                     [addictive, stressful, time, waster]
4        [steam, comment, section, like, every, comment...
5        [like, gathering, sneak, around, arc, pvp, par...
6        [well, make, game, every, time, hop, experienc...
7        [think, would, sweaty, honestly, somehow, stop...
                               ...                        
69256                                        [hades, home]
69257    [unfortunately, recommend, game, devs, deliver...
69258           [enjoy, soulstone, survivor, hades, enjoy]
69259    [great, game, overall, really, enjoy, hades, l...
69260    [game, pretty, good, first, difficulty, become...
Name: review_lemma_NLTK_final, Length: 48713, dtype: object

## restringing statuses

In [49]:
# create a text column with tagged tokens
review_df['review_cleaned'] = review_df['review_lemma_NLTK_final'].apply (lambda row: ' '.join(str(x) for x in row))
# check the final text
print(review_df['review_cleaned'].head(3).values)

['addictive stressful time waster'
 'steam comment section like every comment section game expect get flame negative review lame pvp ruin solo play extraction shooter devolve see genre unpopular maybe squad mode well consistent group hard say fix much tension make game interesting possibility nearby raider hostile however game make vulnerable long looting favor first shoot strongly anyone actively try murder every player meet massive disadvantage run cool pvp guy even cheap free weapon game heavily favor ambush heavy shield little mitigate minimum feel like deal effective shield damage require form kit investment behalf attacker obviously strategy help many either expensive tedious sneak everywhere never loot anything area open line sight neither terribly fun booby trap every area loot want shooter make unpleasant fact replace anything basic loadout involve obnoxious amount time menu craft equipping let alone expensive limited availability blueprint stock item plus goal maximize loot m

In [52]:
## check rows where the cleaned review is empty
print("Review count before:", review_df.shape)

review_df = review_df[review_df["review_cleaned"].str.len() > 0]
print("Review count after:", review_df.shape)
## We didn't completely clear out any entries so we don't need to worry about this so much

Review count before: (48713, 8)
Review count after: (48662, 8)


## Lexical Diversity

In [53]:
review_df['tokens_text'] = review_df['review_tokens'].apply(nltk.text.Text) # put tokens into text form before calculating lexical diversity
review_df['tokens_text']

0            (Addictive, ., Stressful, ., Time, waster, .)
4        (If, the, Steam, comments, section, is, like, ...
5        (I, like, the, gathering, and, sneaking, aroun...
6        (Very, well, made, game, ,, every, time, I, ho...
7        (I, thought, this, would, be, too, sweaty, for...
                               ...                        
69256                       (We, have, Hades, at, home, :)
69257    (Unfortunately, can, not, recommend, the, game...
69258    (if, you, enjoy, soulstone, survivors, or, had...
69259    (Great, game, overall, !, I, really, enjoy, th...
69260    (The, game, is, pretty, good, for, some, of, t...
Name: tokens_text, Length: 48662, dtype: object

In [54]:
def lexical_diversity(text):
  """
  A measure of the lexical richness of the text
  """
  return len(text)/len(set(text))

In [55]:
review_df['lexical_diversity'] = review_df['tokens_text'].apply(lexical_diversity) ## calculate lexical diversity
review_df['lexical_diversity']

0        1.400000
4        1.775701
5        1.357143
6        1.066667
7        1.259259
           ...   
69256    1.000000
69257    1.177778
69258    1.133333
69259    1.233333
69260    1.105263
Name: lexical_diversity, Length: 48662, dtype: float64

## Spellchecking

In [56]:
!pip install pyspellchecker

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 8.2 MB/s  0:00:00 eta 0:00:01


In [57]:
## import a spellchecker
from spellchecker import SpellChecker
spell = SpellChecker()

In [58]:
review_df['potential_misspells'] = review_df['review_tokens'].apply(spell.unknown) ## store unknown words as potential misspells
review_df['potential_misspells']

0                                               {}
4        {gameplay, pvp, )., loadout, .", griefer}
5                                            {pvp}
6                                               {}
7                                            {isn}
                           ...                    
69256                                           {}
69257                    {promosed, devs, dlc, og}
69258                              {ll, soulstone}
69259                                      {op, ’}
69260                                           {}
Name: potential_misspells, Length: 48662, dtype: object

## review Length

In [59]:
review_df['review_length'] = review_df['review_tokens'].apply(len) ## calculate original review length
review_df['review_length']

0          7
4        380
5         57
6         16
7         68
        ... 
69256      6
69257     53
69258     17
69259     74
69260     21
Name: review_length, Length: 48662, dtype: int64

In [60]:
review_df['cleaned_review_length'] = review_df['review_lemma_NLTK_final'].apply(len) ## calculate review length after lemmatization
review_df['cleaned_review_length']

0          4
4        179
5         21
6         10
7         23
        ... 
69256      2
69257     22
69258      5
69259     33
69260      9
Name: cleaned_review_length, Length: 48662, dtype: int64

## Write results

In [61]:
review_df.head(3)

,recommendationid,Appname,review_text,review_tokens,review_tagged_NLTK,review_lemma_NLTK,review_lemma_NLTK_final,review_cleaned,tokens_text,lexical_diversity,potential_misspells,review_length,cleaned_review_length
0,212664845,ARC Raiders,Addictive. Stressful. Time waster.,"[Addictive, ., Stressful, ., Time, waster, .]","[(Addictive, NNP), (., .), (Stressful, NNP), (...","[Addictive, ., Stressful, ., Time, waster, .]","[addictive, stressful, time, waster]",addictive stressful time waster,"(Addictive, ., Stressful, ., Time, waster, .)",1.400000,{},7,4
4,212664705,ARC Raiders,If the Steam comments section is like every ot...,"[If, the, Steam, comments, section, is, like, ...","[(If, IN), (the, DT), (Steam, NNP), (comments,...","[If, the, Steam, comment, section, be, like, e...","[steam, comment, section, like, every, comment...",steam comment section like every comment secti...,"(If, the, Steam, comments, section, is, like, ...",1.775701,"{gameplay, pvp, )., loadout, ."", griefer}",380,179
5,212664692,ARC Raiders,I like the gathering and sneaking around the A...,"[I, like, the, gathering, and, sneaking, aroun...","[(I, PRP), (like, VBP), (the, DT), (gathering,...","[I, like, the, gathering, and, sneak, around, ...","[like, gathering, sneak, around, arc, pvp, par...",like gathering sneak around arc pvp part peopl...,"(I, like, the, gathering, and, sneaking, aroun...",1.357143,{pvp},57,21


In [62]:
## write just the output required for section 1
output_path = 'data/preprocessing_metadata_output.csv'

review_df[['recommendationid', 'Appname', 'lexical_diversity', 'potential_misspells', 'review_length', 'cleaned_review_length']].to_csv(output_path, index=False)

part1_df = pd.read_csv(output_path)
part1_df.head(5)


,recommendationid,Appname,lexical_diversity,potential_misspells,review_length,cleaned_review_length
0,212664845,ARC Raiders,1.400000,set(),7,4
1,212664705,ARC Raiders,1.775701,"{'gameplay', 'pvp', ').', 'loadout', '.""', 'gr...",380,179
2,212664692,ARC Raiders,1.357143,{'pvp'},57,21
3,212664560,ARC Raiders,1.066667,set(),16,10
4,212664471,ARC Raiders,1.259259,{'isn'},68,23


In [65]:
## write the tokenized and cleaned output for use in later sections
output_path = 'data/review_tokenized.csv'
review_df[['recommendationid', 'Appname', 'review_cleaned']].to_csv(output_path, index=False)

review_df = pd.read_csv(output_path)
review_df.head(5)

,recommendationid,Appname,review_cleaned
0,212664845,ARC Raiders,addictive stressful time waster
1,212664705,ARC Raiders,steam comment section like every comment secti...
2,212664692,ARC Raiders,like gathering sneak around arc pvp part peopl...
3,212664560,ARC Raiders,well make game every time hop experience somet...
4,212664471,ARC Raiders,think would sweaty honestly somehow stop playi...


# Part 2: Feature Generation